In [1]:
import pandas as pd
import numpy as np

In [2]:
# CSV 파일 불러오기
df = pd.read_csv("4개년_부산_일별방류수질현황.csv")

In [3]:
# ph 컬럼 제거
df_cleaned = df.drop(columns=['ph'], axis=1)

In [4]:
# 날짜 기반 파생변수 생성
df_cleaned['날짜'] = pd.to_datetime(df_cleaned['날짜'])
df_cleaned['월'] = df_cleaned['날짜'].dt.month

In [5]:
# 여기서 숫자형 강제 변환!
cols_to_fill = ['bod', 'toc', 'ss', 'tn', 'tp']
df_cleaned[cols_to_fill] = df_cleaned[cols_to_fill].apply(pd.to_numeric, errors='coerce')

# 사업단+월 그룹별 중간값 계산
median_values = df_cleaned.groupby(['사업단', '월'])[cols_to_fill].median().reset_index()

df_filled = df_cleaned.copy()

# 각 컬럼 결측치 채우기
for col in cols_to_fill:
    df_filled = df_filled.merge(
        median_values[['사업단', '월', col]].rename(columns={col: f'{col}_median'}),
        on=['사업단', '월'],
        how='left'
    )
    df_filled[col] = df_filled[col].fillna(df_filled[f'{col}_median'])
    df_filled = df_filled.drop(columns=[f'{col}_median'])

# 중간값으로도 채워지지 않은 행 제거
df_final = df_filled.dropna()

In [6]:
## 기간 필터링 및 데이터 품질 점검

# 기준 일수 설정
num_days_full = 1461

# 사업단별 날짜 개수 세기
business_day_counts_full = df_final.groupby('사업단')['날짜'].count().reset_index()
business_day_counts_full.rename(columns={'날짜': '보유일수'}, inplace=True)

# 부족한일수 계산
business_day_counts_full['필요일수'] = num_days_full
business_day_counts_full['부족한일수'] = num_days_full - business_day_counts_full['보유일수']

In [7]:
# 사업단 필터링
df_final = df_final[~df_final['사업단'].isin(["영도사업소", "기장사업소 일광", "동부사업소"])]

In [8]:
# 저장
df_final.to_csv("부산환경공단 일별 방류수 수질정보.csv", index=False, encoding='utf-8-sig', na_rep='null')